# DP Simulation of CIGRE MV without PV Inverters

## Run simulation

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/CIGRE_MV/NEPLAN/CIGRE_MV_no_tapchanger_noLoad1_LeftFeeder_With_LoadFlow_Results/Rootnet_FULL_NE_28J17h'
filename = 'CIGRE-MV'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import dpsimpy

In [ ]:
time_step = 1e-3
final_time = 3.0
steady_state_init = True

In [ ]:
sim_name = 'DP_CIGRE_MV_withoutDG'
sim_name_pf = sim_name + "_Powerflow"

### Powerflow for Initialization

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system_pf = reader.loadCIM(50, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)

logger_pf = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger_pf.log_attribute(node.name() + '.V', 'v', node)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(1.0)
sim_pf.set_final_time(2.0)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(True)

sim_pf.add_logger(logger_pf)
sim_pf.run()


### Dynamic simulation

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
reader2 = dpsimpy.CIMReader(sim_name, dpsimpy.LogLevel.info, dpsimpy.LogLevel.debug)
system_dp = reader2.loadCIM(50, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
system_dp.init_with_powerflow(system_pf)

# log node voltages
logger_dp= dpsimpy.Logger(sim_name)
for node in system_dp.nodes:
    logger_dp.log_attribute(node.name() + '.V', 'v', node)

# log line and load currents
for comp in system_dp.components:
    if isinstance(comp, dpsimpy.dp.ph1.PiLine):
        logger_dp.log_attribute(comp.name() + '.I', 'i_intf', comp)
    if isinstance(comp, dpsimpy.dp.ph1.RXLoad):
        logger_dp.log_attribute(comp.name() + '.I', 'i_intf', comp)

sim_dp = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim_dp.set_system(system_dp)
sim_dp.set_time_step(time_step)
sim_dp.set_final_time(final_time)
sim_dp.set_domain(dpsimpy.Domain.DP)
sim_dp.set_solver(dpsimpy.Solver.MNA)
sim_dp.do_steady_state_init(steady_state_init)
sim_dp.do_init_from_nodes_and_terminals(True)

sim_dp.add_logger(logger_dp)
sim_dp.run()

## Powerflow

### Read results

In [ ]:
modelName = 'DP_CIGRE_MV_withoutDG_Powerflow'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_powerflow = read_timeseries_csv(dpsim_result_file)

### Plot results

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim_powerflow.items():
    if ts_name != 'N0.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### Node voltages

In [ ]:
for ts_name, ts_obj  in ts_dpsim_powerflow.items():
    print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

## Dynamic Phasor

### Read results

In [ ]:
modelName = 'DP_CIGRE_MV_withoutDG'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

## Node voltages

### Plot results

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name != 'N0.V' and ts_name[-2:] == '.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend(loc='upper right')
plt.show()

### At first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.V':
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### At last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.V':
        print(ts_name + ': ' + str(ts_obj.abs().values[-1:]) + ', ' + str(ts_obj.phase().values[-1:]))

## Load currents

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.I' and ts_name[:4] == 'LOAD':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.xlim([0,0.01])
plt.legend(loc='upper right')
plt.show()

### At first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.I' and ts_name[:4] == 'LOAD':
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### At last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.I' and ts_name[:4] == 'LOAD':
        print(ts_name + ': ' + str(ts_obj.abs().values[-1:]) + ', ' + str(ts_obj.phase().values[-1:]))

### Load Power Calculations

In [ ]:
load_I_3_s = ts_dpsim['N3.V'].values*np.conj(ts_dpsim['LOAD-I-3.I'].values)  # ref from CIM: P=225250, Q=139597
load_H_11_s = ts_dpsim['N11.V'].values*np.conj(ts_dpsim['LOAD-H-11.I'].values) # ref from CIM: P=329800, Q=82656

### At first timestep

In [ ]:
print('Load-I-3.S' + ': ' + str(load_I_3_s[0]))
print('Load-H-11.S' + ': ' + str(load_H_11_s[0]))

### At last timestep

In [ ]:
print('Load-I-3.S' + ': ' + str(load_I_3_s[-1]))
print('Load-H-11.S' + ': ' + str(load_H_11_s[-1]))